In [4]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import os
import urllib.parse
from dataclasses import dataclass


In [6]:
ROUTING_API_KEY = '5b3ce3597851110001cf624834ddb0fae46447969085ea7efe629c8f'
COUNTRY = "DE"


@dataclass
class GeoCode:
    latitude: float
    longitude: float

    def __str__(self):
        return f"{self.latitude},{self.longitude}"


def get_coordinates(address: str):

    converted_address = urllib.parse.quote(address)

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    }
    url = f'https://api.openrouteservice.org/geocode/search?api_key={ROUTING_API_KEY}&text={converted_address}&boundary.country={COUNTRY}&size=1'
    response = requests.get(url, headers=headers)
    data = response.json()

    if "features" in data and len(data["features"]) > 0:
        coordinates = data["features"][0]["geometry"]["coordinates"]
        latitude, longitude = coordinates
        return GeoCode(latitude, longitude)
    else:
        return None
    

@dataclass
class Route:
    distance_km: float
    duration_mins: float

    def __str__(self):
        return f"{self.distance_km} km, {self.duration_mins} mins"


def get_route(start, end): 
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={ROUTING_API_KEY}&start={start}&end={end}"


    response = requests.get(url)
    data = response.json()

    if "features" in data and len(data["features"]) > 0:
        duration = data["features"][0]["properties"]["segments"][0]["duration"]
        distance = data["features"][0]["properties"]["segments"][0]["distance"]
        return Route(distance_km=distance/1000, duration_mins=duration/60)
    else:
        return None

In [21]:
start = get_coordinates("Park Babelsberg 14, 14482 Potsdam, Germany")
end = get_coordinates("Freunde des Mauerparks e.V, Bernauer Str. 63, 13355 Berlin, Germany")
route = get_route(start, end)

print(start)
print(end)
print(route)

13.098967,52.406464
13.402746,52.540939
31.789 km, 51.855000000000004 mins


In [28]:
def get_charging_points(pos1: GeoCode, pos2: GeoCode):

    bbox = (
        min(pos1.latitude, pos2.latitude), 
        min(pos1.longitude, pos2.longitude), 
        max(pos1.latitude, pos2.latitude),
        max(pos1.longitude, pos2.longitude)
    )

    # Construct query
    query = f'''
[out:json];
(
node["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
way["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
relation["amenity"="charging_station"]({bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]});
);
out center;
    '''

    # Make the API request
    overpass_url = "https://z.overpass-api.de/api/interpreter"
    response = requests.post(overpass_url, data=query)

    # visualize results
    charging_stations = []

    if response.status_code == 200:
        parsed_data = response.json()
        
        for element in parsed_data['elements']:
            if element.get('tags', {}).get('amenity') == 'charging_station':
                if element['type'] == 'node':
                    lat = element.get('lat', None)
                    lon = element.get('lon', None)
                elif element['type'] == 'way' and 'center' in element:
                    lat = element['center'].get('lat', None)
                    lon = element['center'].get('lon', None)
                else:
                    lat = 'Unknown'
                    lon = 'Unknown'
                name = element.get('tags', {}).get('name', 'Unknown Name')
                address = element.get('tags', {}).get('address', 'Unknown Address')
                charging_stations.append({
                    'name': name,
                    'address': address,
                    'lat': lat,
                    'lon': lon
                })
        return charging_stations
    else:
        raise RuntimeError(f"Request failed with status code {response.status_code}")
    

#charging_stations = get_charging_points(GeoCode(42.35, -71.15), GeoCode(42.40, -71.10))
charging_stations = get_charging_points(GeoCode(42, -52), GeoCode(14, 53))
charging_stations

[{'name': 'Unknown Name',
  'address': 'Unknown Address',
  'lat': 28.4592093,
  'lon': -16.2568532},
 {'name': 'LSB-00050',
  'address': 'Unknown Address',
  'lat': 38.7407131,
  'lon': -9.1864577},
 {'name': 'LSB-00032',
  'address': 'Unknown Address',
  'lat': 38.7781734,
  'lon': -9.0975198},
 {'name': 'LSB-00033',
  'address': 'Unknown Address',
  'lat': 38.769902,
  'lon': -9.096762},
 {'name': 'LSB-00064',
  'address': 'Unknown Address',
  'lat': 38.74894,
  'lon': -9.13805},
 {'name': 'LSB-00035',
  'address': 'Unknown Address',
  'lat': 38.7605598,
  'lon': -9.0953221},
 {'name': 'LSB-00039',
  'address': 'Unknown Address',
  'lat': 38.7630416,
  'lon': -9.0968087},
 {'name': 'CSC-00007',
  'address': 'Unknown Address',
  'lat': 38.6937993,
  'lon': -9.3720115},
 {'name': 'LSB-00036',
  'address': 'Unknown Address',
  'lat': 38.77293,
  'lon': -9.0925},
 {'name': 'CSC-00008',
  'address': 'Unknown Address',
  'lat': 38.6906776,
  'lon': -9.3570496},
 {'name': 'LSB-00034 - Alti

In [24]:
start, end

(GeoCode(latitude=13.098967, longitude=52.406464),
 GeoCode(latitude=13.402746, longitude=52.540939))